In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [6]:
# Check GPU availability
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU'))}")
print(f"GPU Devices: {tf.config.list_physical_devices('GPU')}")


TensorFlow version: 2.20.0
GPU Available: 0
GPU Devices: []


In [7]:
def load_and_preprocess_data():
    """Load CIFAR-10 dataset and apply preprocessing."""
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    
    # Normalize pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0
    
    # Convert labels to categorical
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    
    print(f"Training set shape: {x_train.shape}")
    print(f"Test set shape: {x_test.shape}")
    
    return x_train, y_train, x_test, y_test


In [8]:
# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']


In [9]:
x_train, y_train, x_test, y_test = load_and_preprocess_data()


Training set shape: (50000, 32, 32, 3)
Test set shape: (10000, 32, 32, 3)


## VGG-13 Architecture

VGG-13 consists of:
- 10 convolutional layers (in 5 blocks)
- 3 fully connected layers
- Total: 13 weight layers


In [10]:
def build_vgg13(input_shape=(32, 32, 3), num_classes=10):
    """Build VGG-13 architecture adapted for CIFAR-10."""
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # Block 1: 2 conv layers with 64 filters
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), strides=(2, 2)),
        layers.Dropout(0.2),
        
        # Block 2: 2 conv layers with 128 filters
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), strides=(2, 2)),
        layers.Dropout(0.3),
        
        # Block 3: 2 conv layers with 256 filters
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), strides=(2, 2)),
        layers.Dropout(0.4),
        
        # Block 4: 2 conv layers with 512 filters
        layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), strides=(2, 2)),
        layers.Dropout(0.4),
        
        # Block 5: 2 conv layers with 512 filters
        layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2), strides=(2, 2)),
        layers.Dropout(0.5),
        
        # Fully connected layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model


In [11]:
# Build model
model = build_vgg13()
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 4, 4, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 9,951,306 (37.96 MB)

 Trainable params: 9,943,370 (37.93 MB)

 Non-trainable params: 7,936 (31.00 KB)

## Model Compilation and Training


In [12]:
# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [13]:
# Data augmentation for better generalization
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)


In [14]:
# Callbacks for training optimization
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)


In [16]:
# Train model
batch_size = 64
epochs = 50

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[reduce_lr, early_stop],
    verbose=1
)


Epoch 1/50
  2/782 ━━━━━━━━━━━━━━━━━━━━ 9:24 723ms/step - accuracy: 0.1836 - loss: 2.8558 

KeyboardInterrupt: 

## Training History Visualization


In [ ]:
def plot_training_history(history):
    """Plot training and validation accuracy/loss curves."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Accuracy', fontsize=12)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Loss plot
    ax2.plot(history.history['loss'], label='Train Loss', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)


## Model Evaluation


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\n{'='*50}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"{'='*50}\n")


In [ ]:
# Get predictions
y_pred = model.predict(x_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)


In [ ]:
# Classification report
print("Classification Report:")
print("="*70)
print(classification_report(y_true_classes, y_pred_classes, 
                          target_names=class_names, digits=4))


## Confusion Matrix


In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    """Plot confusion matrix heatmap."""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix - VGG-13 on CIFAR-10', fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(y_true_classes, y_pred_classes, class_names)


## Predictions Visualization


In [ ]:
def visualize_predictions(x_test, y_true, y_pred, class_names, n_images=16):
    """Visualize test images with true and predicted labels."""
    fig, axes = plt.subplots(4, 4, figsize=(14, 14))
    axes = axes.ravel()
    
    # Select random indices
    indices = np.random.choice(len(x_test), n_images, replace=False)
    
    for i, idx in enumerate(indices):
        axes[i].imshow(x_test[idx])
        axes[i].axis('off')
        
        true_label = class_names[y_true[idx]]
        pred_label = class_names[y_pred[idx]]
        
        # Color: green if correct, red if incorrect
        color = 'green' if y_true[idx] == y_pred[idx] else 'red'
        
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}',
                         fontsize=10, color=color, fontweight='bold')
    
    plt.suptitle('VGG-13 Predictions on CIFAR-10 Test Set', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

visualize_predictions(x_test, y_true_classes, y_pred_classes, class_names)


## Analysis of Correct and Incorrect Predictions


In [ ]:
def show_correct_and_incorrect_predictions(x_test, y_true, y_pred, y_pred_prob, class_names):
    """Display examples of correct and incorrect predictions."""
    # Find correct and incorrect predictions
    correct_idx = np.where(y_true == y_pred)[0]
    incorrect_idx = np.where(y_true != y_pred)[0]
    
    print(f"Total correct predictions: {len(correct_idx)} ({len(correct_idx)/len(y_true)*100:.2f}%)")
    print(f"Total incorrect predictions: {len(incorrect_idx)} ({len(incorrect_idx)/len(y_true)*100:.2f}%)")
    
    # Show some correct predictions
    fig, axes = plt.subplots(2, 5, figsize=(16, 7))
    fig.suptitle('Correct Predictions (High Confidence)', fontsize=16, fontweight='bold')
    
    # Sort correct predictions by confidence
    correct_confidences = [y_pred_prob[idx][y_pred[idx]] for idx in correct_idx]
    sorted_correct_idx = correct_idx[np.argsort(correct_confidences)[::-1][:10]]
    
    for i, idx in enumerate(sorted_correct_idx):
        ax = axes[i//5, i%5]
        ax.imshow(x_test[idx])
        ax.axis('off')
        confidence = y_pred_prob[idx][y_pred[idx]]
        ax.set_title(f'{class_names[y_true[idx]]}\nConf: {confidence:.3f}',
                    fontsize=10, color='green', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Show some incorrect predictions
    if len(incorrect_idx) > 0:
        fig, axes = plt.subplots(2, 5, figsize=(16, 7))
        fig.suptitle('Incorrect Predictions', fontsize=16, fontweight='bold')
        
        selected_incorrect = incorrect_idx[np.random.choice(len(incorrect_idx), 
                                                           min(10, len(incorrect_idx)), 
                                                           replace=False)]
        
        for i, idx in enumerate(selected_incorrect):
            ax = axes[i//5, i%5]
            ax.imshow(x_test[idx])
            ax.axis('off')
            true_label = class_names[y_true[idx]]
            pred_label = class_names[y_pred[idx]]
            confidence = y_pred_prob[idx][y_pred[idx]]
            ax.set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}',
                        fontsize=9, color='red', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

show_correct_and_incorrect_predictions(x_test, y_true_classes, y_pred_classes, y_pred, class_names)


## Per-Class Accuracy Analysis


In [ ]:
# Calculate per-class accuracy
def plot_per_class_accuracy(y_true, y_pred, class_names):
    """Plot accuracy for each class."""
    class_accuracies = []
    
    for i in range(len(class_names)):
        class_mask = y_true == i
        class_correct = np.sum((y_true[class_mask] == y_pred[class_mask]))
        class_total = np.sum(class_mask)
        class_accuracy = class_correct / class_total if class_total > 0 else 0
        class_accuracies.append(class_accuracy)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(range(len(class_names)), class_accuracies, color='steelblue', alpha=0.8)
    
    # Add value labels on bars
    for i, (bar, acc) in enumerate(zip(bars, class_accuracies)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{acc:.3f}',
               ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Class', fontsize=12, fontweight='bold')
    ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
    ax.set_title('Per-Class Accuracy - VGG-13 on CIFAR-10', fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right')
    ax.set_ylim([0, 1.1])
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\nPer-Class Accuracy Summary:")
    print("="*50)
    for name, acc in zip(class_names, class_accuracies):
        print(f"{name:12s}: {acc:.4f} ({acc*100:.2f}%)")
    print("="*50)
    print(f"Mean Accuracy: {np.mean(class_accuracies):.4f} ({np.mean(class_accuracies)*100:.2f}%)")

plot_per_class_accuracy(y_true_classes, y_pred_classes, class_names)


## Model Saving


In [ ]:
# Save model
model.save('vgg13_cifar10.h5')
print("Model saved successfully as 'vgg13_cifar10.h5'")
